# Running Custom Jupyter in K8s

This notebook will walk through using Kubernetes with Jupyter

## Step 0: Prerequisites

1. You must have an NRP account
2. You must have been added to a Nautilus namespace
3. You must have your NRP config in the `~/.kube` directory. 



## Step 1: Building the Container

The first step is to build a container with whatever custom software you need:



### Dockerfile
```dockerfile
FROM quay.io/jupyter/scipy-notebook:latest

RUN pip install geopandas shapely geoplot
```

### Building the Container Image

```bash
docker build -f ./docker/Jupyter.Dockerfile -t mynamespace/geo-jupyter:latest ./docker
```


### Pushing to Registry
```bash
docker push mynamespace/geo-jupyter:latest
```


All of these steps have been done for us and there is a container available at:
```
jalexhurt/geo-jupyter:latest
```



## Step 2: Building the Pod Specification YAML

We now have everything we need to run our Jupyter Pod. Let's create our YAML File:

```yaml
apiVersion: v1
kind: Pod
metadata:
  name: {{pod_name}}
spec:
  automountServiceAccountToken: false
  affinity:
    nodeAffinity:
      requiredDuringSchedulingIgnoredDuringExecution:
        nodeSelectorTerms:
        - matchExpressions:
          - key: topology.kubernetes.io/region
            operator: In
            values:
            - us-central 
  containers:
    - name: {{pod_name}}
      image: jalexhurt/geo-jupyter:latest
      env:
          - name: NOTEBOOK_ARGS
            value: "--ip 0.0.0.0 --port 8888 --allow-root"
      resources:
        limits:
          memory: 12Gi
          cpu: 2
        requests:
          memory: 10Gi
          cpu: 2

```

Let's use `jinja2` to fill in the missing values in our Template:

In [ ]:
from jinja2 import Template

# read in the template
with open('../yaml/jupyter_pod_template.yml') as file_:
    template = Template(file_.read())

Replace the arguments to the `render` function with the appropriate values:

In [ ]:
pod_spec = template.render(
    pod_name=input("Pod Name: ")
)

print(pod_spec)

Now, let's save it to disk:

In [ ]:
with open("./jupyter_pod.yml", "w") as file:
    file.write(pod_spec)

## Step 3: Start the Pod

Run the cell below to start the pod:

In [ ]:
! kubectl create -f ./jupyter_pod.yml

Run the cell below until your job moves to the `Complete` status. It will go through the stages of: `Pending`, `ContainerCreating`, and `Running`:

In [ ]:
! kubectl get pods

## Step 4: Port Forwarding

From a local machine with KubeCTL installed, we would now run:
```
kubectl port-forward PODNAME 8888:8888
```

And then we could access the Jupyter environment from our local web browser by using the URL in the logs of the Pod:


In [ ]:
! kubectl logs PODNAME | grep 127.0.0.1

## Step 5: Delete the Job and the Pod

The final step is to delete the job we ran the pod we spawned. Please change `JOBNAME` and `PODNAME` below to the appropriate name:

In [ ]:
! kubectl delete pod PODNAME